In [0]:
import pyspark.sql.functions as F
import os

STORAGE_ACCOUNT = os.getenv('STORAGE_ACCOUNT')
STORAGE_ACCOUNT_KEY = os.getenv('STORAGE_ACCOUNT_KEY')
spark.conf.set(STORAGE_ACCOUNT, STORAGE_ACCOUNT_KEY)


In [0]:
bangladesh_df = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("abfss://datasets@tfmstorageacc.dfs.core.windows.net/rape_total_2001_2021-bangladesh.csv")

bangladesh_df.display()

This one is pretty clean already we just need a few adjustments

In [0]:
bangladesh_df_melted = bangladesh_df.drop(
    'Unidentified_age'
).melt(
    ids=['Years'],
    values=['Total_victims', 'children', 'women'],
    variableColumnName='age_group',
    valueColumnName='total_cases',
)
bangladesh_df_melted.display()

now change column names and add country information

In [0]:
age_group_dict = {
    'children':'child',
    'women':'adult',
    'Total_victims':'any'
}
bangladesh_df_melted.replace(
    age_group_dict,
    subset=['age_group'],
).withColumn(
    'country', F.lit('bangladesh'),
).select(
    F.col('Years').alias('year'),
    F.col('age_group'),
    F.col('total_cases'),
    F.col('country'),
).display()